### Imports

In [1]:
import numpy as np
import os
from datetime import datetime
import sys

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, MaxPooling2D, TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import sklearn as sk
from keras import regularizers
import platform


print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.3.1-arm64-arm-64bit
Tensor Flow Version: 2.9.0
Keras Version: 2.9.0

Python 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:29) [Clang 14.0.6 ]
Scikit-Learn 1.2.0
GPU is available


##  Load frames

### Old dataset

In [6]:
drowsines_levels = np.array(["0", "5", "10"])
DATA_PATH = os.path.join('./Dataset')
label_map = {label: num for num, label in enumerate(drowsines_levels)}

files, labels = [], []

for video_index in range(1,11):
  for level in drowsines_levels:
    window = []      
    for frame in range(1, 4001, 2):
      file_path = os.path.join(DATA_PATH, str(video_index), level, (str(frame) + ".npy"))
      if os.path.exists(file_path):
        res = np.load(file_path)        
        window.append(res)
      else:
        print(f"File at {file_path} doesn't exists")
    files.append(window)
    labels.append(label_map[level])
  print(f"File number: {video_index} processed")

File number: 1 processed
File number: 2 processed
File number: 3 processed
File number: 4 processed
File number: 5 processed
File number: 6 processed
File number: 7 processed
File number: 8 processed
File number: 9 processed
File number: 10 processed


In [7]:
# Check loaded files shape
np.array(files).shape

(30, 2000, 1434)

### New dataset


#### Load new dataset pooling

In [ ]:
from multiprocessing import Pool

def load_data(DATA_PATH, drowsines_levels):
    label_map = {label: num for num, label in enumerate(drowsines_levels)}
    files, labels = [], []
    pool = Pool(processes=4)
    for level in drowsines_levels:
        for video_index in range(1, 90):
            file_paths = [os.path.join(DATA_PATH, level, str(video_index), (str(frame) + ".npy")) for frame in range(1, 1001)]
            results = pool.map(process_file, file_paths)
            window = [result for result in results if result is not None]
            files.append(window)
            labels.append(label_map[level])
            print(f"File number: {video_index} processed for level {level}")
    return files, labels

def process_file(file_path):
    if os.path.exists(file_path):
        res = np.load(file_path)
        return res
    else:
        print(f"File at {file_path} doesn't exist")

drowsines_levels = np.array(["Sleepy", "NonSleepy"])
DATA_PATH = os.path.join('./Extracted_Values_Bigger_Confidence')
files, labels = load_data(DATA_PATH, drowsines_levels)

#### Load new dataset standard

In [8]:
drowsines_levels = np.array(["Sleepy", "NonSleepy"])
DATA_PATH = os.path.join('./Extracted_Values_Bigger_Confidence')
label_map = {label: num for num, label in enumerate(drowsines_levels)}

files, labels = [], []

for level in drowsines_levels:
    for video_index in range(1,90):
        window = []      
        for frame in range(1, 1101):
            file_path = os.path.join(DATA_PATH, level, str(video_index), (str(frame) + ".npy"))
            if os.path.exists(file_path):
                res = np.load(file_path)        
                window.append(res)
            else:
                print(f"File at {file_path} doesn't exists")
        files.append(window)
        labels.append(label_map[level])
        print(f"File number: {video_index} processed for level {level}")

# print(f"Processed! Current dataset shape: {np.array(files).shape}")

File number: 1 processed for level Sleepy
File number: 2 processed for level Sleepy
File number: 3 processed for level Sleepy
File number: 4 processed for level Sleepy
File number: 5 processed for level Sleepy
File number: 6 processed for level Sleepy
File number: 7 processed for level Sleepy
File number: 8 processed for level Sleepy
File number: 9 processed for level Sleepy
File number: 10 processed for level Sleepy
File number: 11 processed for level Sleepy
File number: 12 processed for level Sleepy
File number: 13 processed for level Sleepy
File number: 14 processed for level Sleepy
File number: 15 processed for level Sleepy
File number: 16 processed for level Sleepy
File number: 17 processed for level Sleepy
File number: 18 processed for level Sleepy
File number: 19 processed for level Sleepy
File number: 20 processed for level Sleepy
File number: 21 processed for level Sleepy
File number: 22 processed for level Sleepy
File number: 23 processed for level Sleepy
File number: 24 proc

#### Load new dataset fancy progress

In [ ]:
def processDataset():
    drowsines_levels = np.array(["Sleepy", "NonSleepy"])
    DATA_PATH = os.path.join('./Extracted_Values_Bigger_Confidence')
    label_map = {label: num for num, label in enumerate(drowsines_levels)}

    files, labels = [], []

    for level in drowsines_levels:
        for video_index in range(1,90):
            window = []      
            for frame in range(1, 1001):
                file_path = os.path.join(DATA_PATH, level, str(video_index), (str(frame) + ".npy"))
                if os.path.exists(file_path):
                    res = np.load(file_path)        
                    window.append(res)
                else:
                    print(f"File at {file_path} doesn't exists")
            files.append(window)
            labels.append(label_map[level])
            progress_percentage = (video_index / 89) * 100  # Calculate the percentage progress
            status = f"Progress: {progress_percentage:.2f}% | Level: {level}"
            print(status, end="\r")  # Overwrite the previous status

    print(f"Processed! Current dataset shape: {np.array(files).shape}")

In [9]:
np.array(files).shape

(178, 1100, 1434)

## Model training

#### Define model

In [10]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1100, 1434)))
model.add(LSTM(128, return_sequences=True, activation='tanh')) 
model.add(LSTM(32, return_sequences=False, activation='tanh')) 
model.add(Dense(64, activation='softmax', kernel_regularizer='l2')) 
model.add(Dropout(0.1))
model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

model.summary()
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 1100, 64)          383744    
                                                                 
 lstm_4 (LSTM)               (None, 1100, 128)         98816     
                                                                 
 lstm_5 (LSTM)               (None, 32)                20608     
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 505,410
Trainable params: 505,410
Non-tr

#### Train Test Split Validation

In [14]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_dir = os.path.join(f'Logs/Input_1100_Frames/Logs_train_test_split_batch_300')
tb_callback = TensorBoard(log_dir=log_dir)
es = EarlyStopping(monitor='val_loss', patience=50)

model.fit(X_train, y_train, epochs=500, batch_size=300, shuffle=True, callbacks=[tb_callback, es], validation_split = 0.2) 

model.save(f'Models/input_1100_b300_drowsines_levels_train_test_split_${date_string}.h5')

Epoch 1/500
1/1 [==============================] - 14s 14s/step - loss: 0.7172 - accuracy: 0.5044 - val_loss: 0.7203 - val_accuracy: 0.4138
Epoch 2/500
1/1 [==============================] - 5s 5s/step - loss: 0.7162 - accuracy: 0.4867 - val_loss: 0.7197 - val_accuracy: 0.4138
Epoch 3/500
1/1 [==============================] - 2s 2s/step - loss: 0.7156 - accuracy: 0.5044 - val_loss: 0.7192 - val_accuracy: 0.4138
Epoch 4/500
1/1 [==============================] - 2s 2s/step - loss: 0.7163 - accuracy: 0.4867 - val_loss: 0.7186 - val_accuracy: 0.4138
Epoch 5/500
1/1 [==============================] - 2s 2s/step - loss: 0.7160 - accuracy: 0.4867 - val_loss: 0.7181 - val_accuracy: 0.4138
Epoch 6/500
1/1 [==============================] - 2s 2s/step - loss: 0.7151 - accuracy: 0.4867 - val_loss: 0.7176 - val_accuracy: 0.4138
Epoch 7/500
1/1 [==============================] - 3s 3s/step - loss: 0.7154 - accuracy: 0.4867 - val_loss: 0.7170 - val_accuracy: 0.4138
Epoch 8/500
1/1 [===============

In [18]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_dir = os.path.join(f'Logs/BatchSizesTuning/Logs_train_test_split_batch_150')
tb_callback = TensorBoard(log_dir=log_dir)
es = EarlyStopping(monitor='val_loss', patience=50)

model.fit(X_train, y_train, epochs=500, batch_size=150, shuffle=True, callbacks=[tb_callback, es], validation_split = 0.2) 

model.save(f'Models/drowsines_levels_train_test_split_${date_string}.h5')

Epoch 1/500
1/1 [==============================] - 11s 11s/step - loss: 0.5059 - accuracy: 0.7434 - val_loss: 0.5816 - val_accuracy: 0.7241
Epoch 2/500
1/1 [==============================] - 3s 3s/step - loss: 0.5037 - accuracy: 0.7611 - val_loss: 0.5864 - val_accuracy: 0.7241
Epoch 3/500
1/1 [==============================] - 2s 2s/step - loss: 0.5023 - accuracy: 0.7699 - val_loss: 0.6009 - val_accuracy: 0.7241
Epoch 4/500
1/1 [==============================] - 2s 2s/step - loss: 0.4973 - accuracy: 0.7788 - val_loss: 0.6034 - val_accuracy: 0.7241
Epoch 5/500
1/1 [==============================] - 2s 2s/step - loss: 0.4765 - accuracy: 0.7788 - val_loss: 0.6030 - val_accuracy: 0.7241
Epoch 6/500
1/1 [==============================] - 2s 2s/step - loss: 0.4877 - accuracy: 0.7522 - val_loss: 0.6057 - val_accuracy: 0.7241
Epoch 7/500
1/1 [==============================] - 2s 2s/step - loss: 0.4954 - accuracy: 0.7434 - val_loss: 0.6015 - val_accuracy: 0.7241
Epoch 8/500
1/1 [===============

In [21]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_dir = os.path.join(f'Logs/BatchSizesTuning/Logs_train_test_split_batch_250')
tb_callback = TensorBoard(log_dir=log_dir)
es = EarlyStopping(monitor='val_loss', patience=50)

model.fit(X_train, y_train, epochs=500, batch_size=250, shuffle=True, callbacks=[tb_callback, es], validation_split = 0.2) 

model.save(f'Models/drowsines_levels_train_test_split_${date_string}.h5')

Epoch 1/500
1/1 [==============================] - 13s 13s/step - loss: 0.4484 - accuracy: 0.8053 - val_loss: 0.4562 - val_accuracy: 0.8276
Epoch 2/500
1/1 [==============================] - 6s 6s/step - loss: 0.4946 - accuracy: 0.7699 - val_loss: 0.4563 - val_accuracy: 0.8276
Epoch 3/500
1/1 [==============================] - 2s 2s/step - loss: 0.4729 - accuracy: 0.7965 - val_loss: 0.4564 - val_accuracy: 0.8276
Epoch 4/500
1/1 [==============================] - 2s 2s/step - loss: 0.4355 - accuracy: 0.8142 - val_loss: 0.4565 - val_accuracy: 0.8276
Epoch 5/500
1/1 [==============================] - 2s 2s/step - loss: 0.4710 - accuracy: 0.8142 - val_loss: 0.4564 - val_accuracy: 0.8276
Epoch 6/500
1/1 [==============================] - 2s 2s/step - loss: 0.4615 - accuracy: 0.7876 - val_loss: 0.4563 - val_accuracy: 0.8276
Epoch 7/500
1/1 [==============================] - 2s 2s/step - loss: 0.4647 - accuracy: 0.7876 - val_loss: 0.4563 - val_accuracy: 0.8276
Epoch 8/500
1/1 [===============

In [23]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_dir = os.path.join(f'Logs/BatchSizesTuning/Logs_train_test_split_batch_275')
tb_callback = TensorBoard(log_dir=log_dir)
es = EarlyStopping(monitor='val_loss', patience=50)

# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1099, 1434)))
# model.add(LSTM(128, return_sequences=True, activation='tanh')) 
# model.add(LSTM(32, return_sequences=False, activation='tanh')) 
# model.add(Dense(32, activation='softmax', kernel_regularizer='l2')) 
# model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

# model.summary()
# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=500, batch_size=275, shuffle=True, callbacks=[tb_callback, es], validation_split = 0.2) 

model.save(f'Models/drowsines_levels_train_test_split_${date_string}.h5')

Epoch 1/500
1/1 [==============================] - 6s 6s/step - loss: 0.4025 - accuracy: 0.8407 - val_loss: 0.6111 - val_accuracy: 0.7241
Epoch 2/500
1/1 [==============================] - 2s 2s/step - loss: 0.3886 - accuracy: 0.8407 - val_loss: 0.6115 - val_accuracy: 0.7241
Epoch 3/500
1/1 [==============================] - 2s 2s/step - loss: 0.3795 - accuracy: 0.8319 - val_loss: 0.6118 - val_accuracy: 0.7241
Epoch 4/500
1/1 [==============================] - 2s 2s/step - loss: 0.3733 - accuracy: 0.8584 - val_loss: 0.6111 - val_accuracy: 0.7241
Epoch 5/500
1/1 [==============================] - 2s 2s/step - loss: 0.3773 - accuracy: 0.8584 - val_loss: 0.6081 - val_accuracy: 0.7241
Epoch 6/500
1/1 [==============================] - 2s 2s/step - loss: 0.3484 - accuracy: 0.8673 - val_loss: 0.6023 - val_accuracy: 0.7241
Epoch 7/500
1/1 [==============================] - 2s 2s/step - loss: 0.3894 - accuracy: 0.8584 - val_loss: 0.6023 - val_accuracy: 0.7241
Epoch 8/500
1/1 [=================

In [22]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_dir = os.path.join(f'Logs/BatchSizesTuning/Logs_train_test_split_batch_300')
tb_callback = TensorBoard(log_dir=log_dir)
es = EarlyStopping(monitor='val_loss', patience=50)

model.fit(X_train, y_train, epochs=500, batch_size=300, shuffle=True, callbacks=[tb_callback, es], validation_split = 0.2) 

model.save(f'Models/drowsines_levels_train_test_split_${date_string}.h5')

Epoch 1/500
1/1 [==============================] - 6s 6s/step - loss: 0.4105 - accuracy: 0.8407 - val_loss: 0.6109 - val_accuracy: 0.7241
Epoch 2/500
1/1 [==============================] - 2s 2s/step - loss: 0.4113 - accuracy: 0.8407 - val_loss: 0.6110 - val_accuracy: 0.7241
Epoch 3/500
1/1 [==============================] - 2s 2s/step - loss: 0.3892 - accuracy: 0.8496 - val_loss: 0.6113 - val_accuracy: 0.7241
Epoch 4/500
1/1 [==============================] - 2s 2s/step - loss: 0.3850 - accuracy: 0.8761 - val_loss: 0.6117 - val_accuracy: 0.7241
Epoch 5/500
1/1 [==============================] - 2s 2s/step - loss: 0.3888 - accuracy: 0.8673 - val_loss: 0.6125 - val_accuracy: 0.7241
Epoch 6/500
1/1 [==============================] - 2s 2s/step - loss: 0.3889 - accuracy: 0.8584 - val_loss: 0.6137 - val_accuracy: 0.7241
Epoch 7/500
1/1 [==============================] - 2s 2s/step - loss: 0.3764 - accuracy: 0.8496 - val_loss: 0.6157 - val_accuracy: 0.7241
Epoch 8/500
1/1 [=================

### Cross validation

In [ ]:
from sklearn.model_selection import KFold

now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)

es = EarlyStopping(monitor='val_loss', patience=50)

log_dir = os.path.join(f'Logs/Logs_{date_string}_CrossValid')
tb_callback = TensorBoard(log_dir=log_dir)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = []
for train, test in kfold.split(X):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1100, 1434)))
    model.add(LSTM(128, return_sequences=True, activation='tanh')) 
    model.add(LSTM(32, return_sequences=False, activation='tanh')) 
    model.add(Dense(64, activation='softmax', kernel_regularizer='l2')) 
    model.add(Dropout(0.1))
    model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

    model.summary()
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(X[train], y[train], epochs=1000, batch_size=250, shuffle=True, callbacks=[tb_callback, es], validation_split=0.2) 

    scores = model.evaluate(X[test], y[test], verbose=0)
    print(f'Accuracy: {scores[1]*100}%')
    cv_scores.append(scores[1])

print(f'Cross-validation accuracy: {np.mean(cv_scores)*100:.2f}% +/- {np.std(cv_scores)*100:.2f}%')
model.save(f'Models/drowsines_levels_cross_validation_${date_string}.h5')

#### Find hyperparameters


In [ ]:
from sklearn.model_selection import KFold

now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)

es = EarlyStopping(monitor='val_loss', patience=50)

log_dir = os.path.join(f'Logs/BatchSizesTuning/Logs_{date_string}_CrossValid')
tb_callback = TensorBoard(log_dir=log_dir)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

batch_sizes = [50, 100, 150, 200]
batch_size_scores = []
cv_scores = []

for batch_size in batch_sizes:

    for train, test in kfold.split(X):
        model = Sequential()
        model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1000, 1434)))
        model.add(LSTM(128, return_sequences=True, activation='tanh')) 
        model.add(LSTM(32, return_sequences=False, activation='tanh')) 
        model.add(Dense(32, activation='softmax', kernel_regularizer='l2')) 
        model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

        model.summary()
        model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

        model.fit(X[train], y[train], epochs=1000, batch_size=batch_size, shuffle=True, callbacks=[tb_callback, es], validation_split=0.2) 

        scores = model.evaluate(X[test], y[test], verbose=0)
        print(f'Accuracy: {scores[1]*100}%')
        cv_scores.append(scores[1])

    cross_validation_accuracy = np.mean(cv_scores)*100
    cross_validation_accuracy_error = np.std(cv_scores)*100
    print(f'Cross-validation accuracy: {cross_validation_accuracy:.2f}% +/- {cross_validation_accuracy_error:.2f}% for batch size: {batch_size}')
    batch_size_scores.append(dict(accuracy = cross_validation_accuracy, accuracy_error = cross_validation_accuracy_error, batch_size_used = batch_size))

In [ ]:
for score in batch_size_scores:
    print(score)

#### Grid search for parameters

In [11]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

drowsines_levels = np.array(["Sleepy", "NonSleepy"])
X = np.array(files)
y = to_categorical(labels).astype(int)
X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
def create_model(batch_size):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1100, 1434)))
    model.add(LSTM(128, return_sequences=True, activation='tanh'))
    model.add(LSTM(32, return_sequences=False, activation='tanh'))
    model.add(Dense(32, activation='softmax', kernel_regularizer='l2'))
    model.add(Dense(drowsiness_levels.shape[0], activation='softmax'))
    
    optimizer = Adam()
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


batch_sizes = [10, 20, 50, 100]

model = KerasClassifier(build_fn=create_model)

param_grid = {'batch_size': batch_sizes, 'epochs': [500]}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(X_train, y_train)

print("Best Batch Size: ", grid_result.best_params_['batch_size'])


/var/folders/xs/2z00tyb10f3cb3gphym50rn00000gp/T/ipykernel_33086/1155999763.py:34: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


ValueError: 
All the 12 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/M374155/miniforge3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/M374155/miniforge3/lib/python3.10/site-packages/keras/wrappers/scikit_learn.py", line 236, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/Users/M374155/miniforge3/lib/python3.10/site-packages/keras/wrappers/scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "/var/folders/xs/2z00tyb10f3cb3gphym50rn00000gp/T/ipykernel_33086/1155999763.py", line 21, in create_

### Other architecture experiments

In [ ]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# es = EarlyStopping(monitor='val_loss', patience=50)

log_dir = os.path.join(f'Logs_15')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1000, 1434)))
model.add(LSTM(128, return_sequences=True, activation='tanh')) 
model.add(TimeDistributed(Dropout(0.1)))
model.add(LSTM(32, return_sequences=False, activation='tanh')) 
model.add(Dense(32, activation='softmax', kernel_regularizer=regularizers.l2(0.008))) 
model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

model.summary()
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=500, batch_size=50, shuffle = True, callbacks=[tb_callback], validation_split = 0.2) 

model.save(f'drowsines_weights_noStop_l2008_drop01_${date_string}.h5')

## Model validation

### Confusion Matrix

In [ ]:
def generateConfusionMatrix():
    yhat = model.predict(X_test)
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    multilabel_confusion_matrix(ytrue, yhat)


### Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Logs/Logs_15/04/2023 21:41:01

## Links

In [ ]:
https://machinelearningmastery.com/use-dropout-lstm-networks-time-series-forecasting/ - LSTM Dropout
https://openai.com/research/how-ai-training-scales - OpenAI odnosnie bathc size
https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594 - hyperparameters

In [ ]:
image_height = 0
image_width = 120